In [1]:
!pip install spotipy --upgrade

In [2]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials as cc
import numpy as np
import pandas as pd
from IPython.display import display
import warnings
import time
import json
from pprint import pprint
from spotipy.oauth2 import SpotifyClientCredentials

In [5]:
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id = '85bc9d29761941f2a0dafdc9aa8a4903',client_secret = 'e1ee091ccf0b44b6a0dc243abb2f1414'),requests_timeout=10, retries=10)

In [4]:
playlists = []
path = 'data'
count = 5
for filename in os.listdir(path):
    count -= 1
    if count < 0:
        break
    f = os.path.join(path, filename)
    if os.path.isfile(f):
        d = json.load(open(f))
        playlists.append(pd.DataFrame(d['playlists']))

playlists = pd.concat(playlists)
playlists = playlists.reset_index(drop=True)
print(playlists.shape)

(5000, 12)


In [5]:
songs = []
for i in range(len(playlists)):
    tracks = playlists.iloc[i].loc['tracks']
    for track in tracks:
        songs.append(track)

songs_df = pd.DataFrame(songs)
print(songs_df.shape)
songs_df.head()

(334487, 8)


,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,Lost Frequencies,spotify:track:2vCtiBvJJZfz773yTfAxPP,spotify:artist:7f5Zgnp2spUuuzKplmRkt7,What Is Love 2016 - Mike Mago Remix,spotify:album:4N1e8k2o2NN932Y8xgZV1p,209882,What Is Love 2016
1,1,John Legend,spotify:track:5WOLZP8KrXiupBjG1SSN5U,spotify:artist:5y2Xq6xcjJb2jVM54GHK3t,Love Me Now - Dave Audé Remix Radio Edit,spotify:album:2CFZOjwtV7B6XzULEwIdHN,219106,Love Me Now (Remixes)
2,2,July Child,spotify:track:0oQDQ9QiqsO63EEBAro8Le,spotify:artist:3Jghk94Gog2dhBAO7ZZaM8,Thinkin of U,spotify:album:2i0oT2HtRN4o3P45zQjel7,194000,Thinkin of U
3,3,Tiësto,spotify:track:7MUS0La2IQ85vJ59fQqtoN,spotify:artist:2o5jDhtHVPhrJdv3cEQ99Z,I Will Be Here - (Wolfgang Gartner Remix),spotify:album:2qY9UxNEbQR2O2ECc3Fd48,237066,Kaleidoscope Remixed
4,4,Joe Stone,spotify:track:3ciyZYofjiqmMUElM5qgGB,spotify:artist:4kwEd1P9j15ZqUVP5zK7Pv,The Party (This Is How We Do It),spotify:album:3SBDe6WBbEXNtjraPxXLOh,185807,The Party (This Is How We Do It)


In [6]:
songs_df = songs_df.drop_duplicates('track_uri')
print(songs_df.shape)

(108870, 8)


In [25]:
#DO NOT RUN THE AUDIO FEATURES HAVE ALREADY BEEN STORED IN THE AUDIO_FEATURES FILE
def run_features():
    sleep_min = 1
    sleep_max = 2
    start_time = time.time()
    request_count = 0
    tracks_w_features = pd.DataFrame()
    for i in range(0,3344):
        if request_count % 13 == 0:
            time.sleep(np.random.uniform(sleep_min, sleep_max))
        else:
            tracks_to_get = songs_df.iloc[request_count:request_count+49]["track_uri"] #get the uri of these songs to get audio features
            tracks = pd.DataFrame(sp.audio_features(tracks_to_get))
            tracks_w_features = tracks_w_features.append(tracks)
        request_count+=49

    #remove useless-categorical columns, can use index to get song information later
    # tracks_w_features = tracks_w_features.drop(columns=['mode','key','time_signature', 'duration_ms','analysis_url','track_href','id','type']).copy()
    warnings.filterwarnings("ignore")
    warnings.filterwarnings(action='once')

    from pathlib import Path
    filepath = Path('audio_features.csv')
    filepath.parent.mkdir(parents=True, exist_ok=True)
    tracks_w_features.to_csv(filepath)

In [26]:
# songs = pd.read_csv('recommender-systems/allsongs.csv')
run_features()



# result_artist = sp.artist(songs_df['artist_uri'][0])
# result_song = sp.audio_features(songs_df['track_uri'][0])
# result_song = sp.track(songs_df['track_uri'][0])
# print(result_artist)
# print()
# print("--------------------------------------------")
# print()
# print(result_song)

In [8]:
data = pd.read_csv('audio_features.csv')
data = data.drop(columns=['Unnamed: 0','0','mode','key','time_signature','analysis_url','track_href','id','type']).copy()
# print(data[data['uri'].str.len() == len('spotify:track:6EpRaXYhGOB3fj4V2uDkMJ')])


print(len(data))
data.head()


101526

In [ ]:
pop = []
sleep_min = 1
sleep_max = 2
start_time = time.time()
request_count = 0
count = 0
# 2337
for i in range(0,2457):
        if request_count % 13 == 0:
            time.sleep(np.random.uniform(sleep_min, sleep_max))
            count += 1
        else:

            tracks_to_get = data.iloc[request_count:request_count+43]["uri"] #get the uri of these songs to get audio features
            track_list = sp.tracks(tracks_to_get)
            # pop.extend(sp.tracks(tracks_to_get)['popularity'])
            for j in track_list['tracks']:
                # print('here')
                if j is None:
                    pop.append(None)
                else:
                    pop.append(j['popularity'])
        request_count+=43
        if i % 100 == 0:
            print(i)
        if i > 2300:
            if i % 2 == 0:
                print(i)

print(count)

0
100
200
300
400
500
600
700
800
900
1000


In [89]:
len(pop)

AttributeError: 'list' object has no attribute 'isnane'

In [57]:
list = songs_df.iloc[request_count:request_count+49]["track_uri"]
h = sp.tracks(list)
for j in h['tracks']:
    if(j is not None):
        print(j['popularity'])


28
17
56
52
46
41
60
30
0
0
8
23
25
10
0
25
0
26
0
0
9
49
46
0
34
17
22
0
33
0
0
44
39
36
23
0
39
0
31
52
0
0
67
0
16
67
22
28
0


/Users/macbook/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
list = data.iloc[request_count:request_count+49]["artist"]